# Linear Regression on Hardware Data
This notebook demonstrates how to generate synthetic hardware data and apply linear regression to predict power consumption based on multiple hardware metrics.

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Set a random seed for reproducibility
np.random.seed(42)

# Generate synthetic data
n_records = 1000
cpu_usage = np.random.uniform(0, 100, n_records)  # CPU usage in percentage
ram_usage = np.random.uniform(0, 100, n_records)  # RAM usage in percentage
disk_io = np.random.uniform(0, 1000, n_records)  # Disk I/O in MB/s
network_in = np.random.uniform(0, 1000, n_records)  # Network input in MB/s
network_out = np.random.uniform(0, 1000, n_records)  # Network output in MB/s
temperature = np.random.uniform(20, 80, n_records)  # Temperature in degrees Celsius
fan_speed = np.random.uniform(1000, 4000, n_records)  # Fan speed in RPM
voltage = np.random.uniform(1.0, 1.5, n_records)  # Voltage in volts
noise = np.random.normal(0, 5, n_records)  # Random noise
power_consumption = (30 + 1.5 * cpu_usage + 0.5 * ram_usage + 0.01 * disk_io +
                     0.02 * network_in + 0.02 * network_out + 0.5 * temperature +
                     0.01 * fan_speed + 10 * voltage + noise)  # Linear relationship with some noise

# Create a DataFrame
data = pd.DataFrame({
    'CPU_Usage': cpu_usage,
    'RAM_Usage': ram_usage,
    'Disk_IO': disk_io,
    'Network_In': network_in,
    'Network_Out': network_out,
    'Temperature': temperature,
    'Fan_Speed': fan_speed,
    'Voltage': voltage,
    'Power_Consumption': power_consumption
})

# Display the first few rows of the data
data.head()


## Split the Data
We will split the data into training and testing sets.

In [ ]:

# Split the data into training and testing sets
X = data[['CPU_Usage', 'RAM_Usage', 'Disk_IO', 'Network_In', 'Network_Out', 'Temperature', 'Fan_Speed', 'Voltage']]
y = data['Power_Consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the train and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


## Train the Linear Regression Model
We will create and train a linear regression model using the training data.

In [ ]:

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Display the coefficients
print(f'Intercept: {model.intercept_}')
print(f'Coefficients: {model.coef_}')


## Make Predictions
We will use the trained model to make predictions on the test data.

In [ ]:

# Make predictions
y_pred = model.predict(X_test)

# Display the first few predictions
y_pred[:5]


## Plot the Results
We will plot the actual vs predicted power consumption values.

In [ ]:

# Plot the results
plt.scatter(y_test, y_pred, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Power Consumption')
plt.ylabel('Predicted Power Consumption')
plt.title('Actual vs Predicted Power Consumption')
plt.show()


## Evaluate the Model
We will evaluate the model using Mean Absolute Error (MAE), Mean Squared Error (MSE), Root Mean Squared Error (RMSE), Mean Absolute Percentage Error (MAPE), R-squared (R²) score, and Adjusted R-squared.

In [ ]:

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Percentage Error: {mape}%')
print(f'R-squared: {r2}')
print(f'Adjusted R-squared: {adj_r2}')


## Check Assumptions of Linear Regression
We will check the assumptions of linear regression to validate the model.

### Linearity
The relationship between the independent and dependent variables should be linear.

In [ ]:

# Linearity
plt.figure(figsize=(10, 6))
for i, col in enumerate(X_train.columns, 1):
    plt.subplot(2, 4, i)
    plt.scatter(X_train[col], y_train, alpha=0.5)
    plt.xlabel(col)
    plt.ylabel('Power_Consumption')
plt.tight_layout()
plt.show()


### Independence
The residuals should be independent (no patterns).

In [ ]:

# Independence
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, y_test - y_pred, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.show()


### Homoscedasticity
The residuals should have constant variance.

In [ ]:

# Homoscedasticity
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, y_test - y_pred, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Homoscedasticity Check')
plt.show()


### Normality of Residuals
The residuals should be normally distributed.

In [ ]:

# Normality of Residuals
plt.figure(figsize=(10, 6))
sns.histplot(y_test - y_pred, kde=True)
plt.xlabel('Residuals')
plt.title('Residuals Distribution')
plt.show()

# Q-Q plot for normality
sm.qqplot(y_test - y_pred, line='45')
plt.title('Q-Q Plot')
plt.show()


## Variance Inflation Factor (VIF)
We will calculate VIF to check for multicollinearity among the independent variables.

In [ ]:

# Variance Inflation Factor (VIF)
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print(vif_data)


## Heatmap
We will plot a heatmap to visualize the correlation between features.

In [ ]:

# Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()


## Correlation Matrix
We will display the correlation matrix.

In [ ]:

# Correlation Matrix
corr_matrix = data.corr()
print(corr_matrix)
